# Swift over the Stripe82

From the Swift Master table -- the Swift list of observations, file `SwiftXrt_master.csv` -- we want to select
the ones the fall inside the Stripe82 region.

In [7]:
import pandas as pd
pd.options.display.max_colwidth = 300

In [8]:
!pwd

/Users/chbrandt/Dropbox/PhD/code_repositories/swift_deepsky/docs/notebook


In [9]:
!ls ../../bin

SwiftXrt_master.csv                module_Sosta_log_to_table.py
conv2sedfile.f                     module_Sum_events_maps.sh
conv2sedfile_compile.sh            object2position.py
countrates                         pipeline.sh
countrates.sh                      select_observations.py
download_queue.sh                  sex2deg.sh
download_swift_asdc.pl             swift_deepsky
download_swift_nasa.sh             swifthard03_20_20_10.dat
module_Sosta_adjust_countrates.awk swiftslope.py
module_Sosta_det2sosta.sh


In [10]:
swiftmastr_table = '../../bin/SwiftXrt_master.csv'

In [5]:
# table_master = pandas.read_csv('SwiftXrt_master.csv', sep=';', header=0, low_memory=False)
# table_master

# from numpy import logical_or,logical_and

# ra_idx = logical_or(table_master['RA'].between(0, 60),
#                     table_master['RA'].between(300, 360))

# dec_idx = table_master['DEC'].between(-1.5, 1.5)

# table_stripe82 = table_master.loc[logical_and(ra_idx, dec_idx)]

# table_stripe82

# table_stripe82.to_csv('SwiftXrt_S82.csv', index=False)

In [1]:
def resolve_name(name):
    '''
    Return ICRS position for given object 'name'

    Input:
     - name : str
        Object designation/name
    Output:
     - position : (float,float)
        Tuple with (RA,Dec) coordinates in degree
    '''
    from astropy.coordinates import get_icrs_coordinates as get_coords
    try:
        icrs = get_coords(name)
        pos = (icrs.ra.value,icrs.dec.value)
    except:
        pos = None
    return pos

In [3]:
#!/usr/bin/env python

import astropy
import pandas
import datetime

import logging
import sys

def select_observations(ra, dec, radius=12, swift_mstr_table='SwiftXrt_master.csv',
                        start_time=None, end_time=None):

    def swift_archive_obs_path(date,obsid):
        '''
        Format 'date' and 'obsid' information for swift archive

        Input:
         - date : datetime string
            For example: "13/06/26 23:59:12"
         - obsid : str or int
            Swift OBSID code, e.g, "49650001"

        Output:
         - Swift archive DATE/OBSID address : string
            For example: "2013_06/00049650001"
        '''
        def extract_date(archive_date):
            '''
            Extract year/month from swift date format

            Example:
            "13/06/26 23:59:12" --> "2013_06"
            '''
            from datetime import datetime
            try:
                # dt = datetime.strptime(archive_date,'%y/%m/%d %H:%M:%S')
                dt = datetime.strptime(archive_date,'%d/%m/%Y')
            except ValueError as e:
                print("ERROR: while processing {}".format(archive_date),file=sys.stderr)
                print("ERROR: {}".format(e),file=sys.stderr)
                return None
            year_month = '{:4d}_{:02d}'.format(dt.year,dt.month)
            return year_month

        dtf = extract_date(date)

        if dtf is None:
            return None
        obs = '{:011d}'.format(obsid)
        return '{}/{}'.format(dtf,obs)

    def conesearch(ra_centroid, dec_centroid, radius,
                    ra_list, dec_list):
        '''
        Return a bool array signaling the entries around centroid

        Input:
         - ra_centroid : float
            Reference position' right ascension, in 'degree'
         - dec_centroid: float
            Reference position' declination, in 'degree'
         - radius : float
            Radius, in 'arcmin', to consider around central position
         - ra_list : list of floats
            List of RA positions (in 'degree') to consider
         - dec_list : list of floats
            List of Dec positions (in 'degree') to consider

        Output:
         - Mask arrays : boolean one-dimensional array
            True for (ra/dec_list) positions within 'radius' arcmin
            from (ra/dec_centroid) reference, Flase otherwise
        '''
        from astropy.coordinates import Angle,SkyCoord
        radius = Angle(radius,unit='arcmin')
        coords = SkyCoord(ra_centroid, dec_centroid, unit='degree')
        # ramin = coords.ra - radius
        # ramax = coords.ra + radius
        # raind = (ra_list > ramin.value) * (ra_list < ramax.value)
        # decmin = coords.dec - radius
        # decmax = coords.dec + radius
        # decind = (dec_list > decmin.value) * (dec_list < decmax.value)
        # ind = raind * decind
        # ra_list = ra_list[ind]
        # dec_list = dec_list[ind]
        coords_search = SkyCoord(ra_list, dec_list, unit='degree')

        match_mask = coords.separation(coords_search) < radius
        return match_mask

    def timefilter(table_master, start_time, end_time):
        """
        Input:
         - 'start_time' and 'end_time' are strings in format 'dd/mm/yyy'
        """
        from datetime import datetime
        import pandas as pd
        import numpy as np
        inds = np.ones(len(table_master)).astype(bool)
        if start_time is not None:
            try:
                dt_sel = datetime.strptime(start_time, '%d/%m/%Y')
                dt_vec = pd.to_datetime(table_master['START_TIME'], format='%d/%m/%Y')
                inds &= dt_vec >= dt_sel
            except:
                print('Given start-time format not understood:', start_time)
                return table_master
        if end_time is not None:
            try:
                dt_sel = datetime.strptime(end_time, '%d/%m/%Y')
                dt_vec = pd.to_datetime(table_master['START_TIME'], format='%d/%m/%Y')
                inds &= dt_vec <= dt_sel
            except:
                print('Given end-time format not understood:', end_time)
                return table_master
        return table_master[inds]

    print("Searching Swift Master table: {}".format(swift_mstr_table))
    print("Searching observations around position: {},{}".format(ra,dec))
    print("Search readius: {}".format(radius))

    import pandas
    table_master = pandas.read_csv(swift_mstr_table, sep=';', header=0, low_memory=False)

    table_radec = table_master[['RA','DEC']]
    match_obs_mask = conesearch(ra, dec, radius=radius,
                                ra_list=table_radec['RA'].values,
                                dec_list=table_radec['DEC'].values)
    table_object = table_master.loc[match_obs_mask]

    # If any time limit was given, filter the observations
    #
    if start_time or end_time:
        table_object = timefilter(table_object, start_time, end_time)

    print("Number of observations found: {:d}".format(len(table_object)))

#     if len(table_object) > 0:
#         archive_addr = table_object.apply(lambda x:swift_archive_obs_path(x['START_TIME'],x['OBSID']), axis=1)
#         print("Observation addresses: {}".format(archive_addr.values))
#     else:
#         archive_addr = pandas.DataFrame()

#     from os.path import isdir,dirname
#     if not isdir(dirname(fileout)):
#         print('Needs to create dir for {}'.format(fileout))
#     table_object.to_csv(fileout, index=False)
#     if not isdir(dirname(obsaddrfile)):
#         print('Needs to create dir for {}'.format(obsaddrfile))
#     archive_addr.to_csv(obsaddrfile, index=False)
#     print("Filtered table written to: {}".format(fileout))

    return table_object

In [14]:
select_observations(ra=6.679183, dec=1.306225, radius=12, swift_mstr_table=swiftmastr_table)

Searching Swift Master table: ../../bin/SwiftXrt_master.csv
Searching observations around position: 6.679183,1.306225
Search readius: 12
Number of observations found: 1


,NAME,ORIG_TARGET_ID,TARGET_ID,RA,DEC,START_TIME,STOP_TIME,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC,ARCHIVE_DATE,PROCESSING_DA,PROCESSING_DATE,PROCESSING_VE
18193,GRB070808,287260,287260,6.757718,1.128446,9/8/2007,9/8/2007,1,1,287260001,287260001,16726,16726,20/8/2007,HEA_21DEC2012_V6.13_SWIFT_REL4.0(BLD29)_14DEC2012_SDCPATCH_15,19/7/2015,3.16.09
